# Data wrangling scripts

## Imports and function definitions

In [10]:
import os
import rasterio as rio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import glob
import shutil
import utils
import tifffile as tiff

%load_ext autoreload
%autoreload 2

def rescale_and_save(in_fn):
    fn = in_fn.replace("10cm", "20cm")
    out_fn_rgb = fn.replace(".tif", "_rgb.tif")
    out_fn_nir = fn.replace(".tif", "_nir.tif")
    
    if not os.path.exists(fn.split("/Ortho")[0]):
        os.makedirs(os.path.join(fn.split("/Ortho")[0], "Ortho"))
        
    r =  rio.open(in_fn)
    downscale_factor = 0.5
    r_arr = r.read(
        out_shape=(
                r.count,
                int(r.height * downscale_factor),
                int(r.width * downscale_factor)
            ),
            resampling=Resampling.bilinear
    )


    type(r_arr)
    # r_arr = np.moveaxis(r_arr, 0, -1)

    for i in range(4):
        max = r_arr[i].max()
        r_arr[i] = (r_arr[i] / max) * 255
    # r_arr = np.flip(r_arr[0:3, ...], axis=0)
    R = r_arr[2].copy()
    G = r_arr[1]
    B = r_arr[0].copy()

    r_arr[0] = R
    r_arr[2] = B

    # scale image transform
    transform = r.transform * r.transform.scale(
        (r.width / r_arr.shape[-1]),
        (r.height / r_arr.shape[-2])
    )

    rgb_profile = r.profile.copy()
    rgb_profile.update(
        driver="GTiff",
        height=r_arr.shape[1],
        width=r_arr.shape[2],
        dtype=r_arr.dtype,
        count=3,
        transform=transform
    )

    nir_profile = rgb_profile.copy()
    nir_profile.update(
        count=1
    )

    new_rgb = rio.open(
        out_fn_rgb,
        'w',
        **rgb_profile
    )
    new_nir = rio.open(
        out_fn_nir,
        'w',
        **nir_profile
    )
    new_rgb.write(r_arr[0:3])
    new_nir.write(np.expand_dims(r_arr[3], 0))

## Downsample 10 cm files to 20 cm with radiometric rescaling and save

In [ ]:
RESCALE = False

if RESCALE:
    dirs = glob.glob("data/*10cm")
    for dir in dirs[1:]:
        fns = glob.glob(os.path.join(dir, "Ortho/*.tif"))
        for fn in fns:
            rescale_and_save(fn)

## Reorganize files

Desired file structure:
- {location}
    - {resolution}
        - Ortho
            - {tilename}
                - rgb
                    - {tilename}_rgb.tif
                - cir
                    - {tilename}_cir.tif
                - labels
                    - {tilename}_labels.tif

In [12]:
REORGANIZE = False

if REORGANIZE:

    DATA_DIR = "data"
    IM_DIRS = ["rgb", "nir", "labels"]
    dirs = glob.glob(f"{DATA_DIR}/*")
    tilenames = []

    for dir in dirs:
        dir = os.path.join(dir, "20cm/Ortho")
        files = glob.glob(f"{dir}/*.tif")
        
        for file in files:
            # Get the tilename
            tilename = os.path.basename(file).rsplit("_", 1)[0]
            tile_dir = os.path.join(dir, tilename)

            if not os.path.exists(tile_dir):
                # Create the tile directory and subdirectories
                # shutil.rmtree(tile_dir)
                os.mkdir(tile_dir)
                for im_dir in IM_DIRS:
                    os.mkdir(os.path.join(tile_dir, im_dir))
            
            for im_dir in IM_DIRS:
                if file.endswith(f"{im_dir}.tif"):
                    shutil.move(file, os.path.join(tile_dir, im_dir))
        

## Sandbox for aggregating and patchifying


In [7]:
data_dir = "data/WA_Kivalina_01_20219703/20cm/Ortho"
dirs = glob.glob(f"{data_dir}/*")
IM_DIRS = ["rgb", "nir", "labels"]
rgb_fns, nir_fns, label_fns = [], [], []

for dir in dirs:
    rgb = glob.glob(f"{dir}/rgb/*.tif")[0]
    nir = glob.glob(f"{dir}/nir/*.tif")[0]
    labels = glob.glob(f"{dir}/labels/*.tif")[0]
    rgb_fns.append(rgb)
    nir_fns.append(nir)
    label_fns.append(labels)

In [45]:
k = 0
rgb_im = tiff.imread(rgb_fns[k])
# nir_im = np.expand_dims(tiff.imread(nir_fns[k]), -1)
r = np.dstack((rgb_im, nir_im))

In [64]:
# Allow division by zero
np.seterr(divide="ignore", invalid="ignore")
r = r.astype(np.float64)
ndvi = (r[..., -1] - r[..., 0]) / (r[..., -1] + r[..., 0])
ndvi[np.isnan(ndvi)] = -1
ndvi = (((ndvi - ndvi.min()) / (ndvi.max() - ndvi.min())) * 255).astype(np.uint16)
ndvi


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 150, 152, 152],
       [  0,   0,   0, ..., 145, 149, 151],
       [  0,   0,   0, ..., 143, 147, 149]], dtype=uint16)

In [48]:
labels = tiff.imread(label_fns[k])

In [72]:
for i in range(len(rgb_fns)):
    im = tiff.imread(rgb_fns[i])
    labels = tiff.imread(label_fns[i])
    print("RGB:", im.shape, "\nLabels:", labels.shape)

RGB: (2500, 2500, 3) 
Labels: (2500, 2501)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2500, 2500, 3) 
Labels: (2500, 2500)
RGB: (2060, 2500, 3) 
Labels: (2060, 2500)
RGB: (2500, 830, 3) 
Labels: (2500, 830)


In [69]:
X_train, y_train, X_test, y_test = utils.prep_data("data/WA_Kivalina_01_20219703/20cm/Ortho", include_nir=False, add_ndvi=False, squash=True)

Number of directories: 23
RGB size: 23
Labels size: 23


ValueError: could not broadcast input array from shape (72,256,256,3) into shape (81,256,256,3)